### 1. 建立資料夾

In [ ]:
import os   #匯入Python的OS模組，用來進行與作業系統相關的操作，例如建立資料夾
upload_dir = "uploaded_docs"  # 指定一個資料夾名稱，用來儲存使用者上傳的檔案
os.makedirs(upload_dir, exist_ok=True) # 建立資料夾，如果 "uploaded_docs" 資料夾不存在就建立它；若已存在，則不會報錯
print(f"請將你的 .txt, .pdf, .docx 檔案放到這個資料夾中： {upload_dir}")
# 印出提示訊息，告訴使用者將要處理的 .txt、.pdf、.docx 檔案放進這個資料夾中

### 2. 更新必要套件並引入

In [ ]:
!pip install -U langchain langchain-community pypdf python-docx sentence-transformers faiss-cpu 

In [ ]:
from langchain_community.document_loaders import TextLoader, PyPDFLoader, UnstructuredWordDocumentLoader
# 從langchain_community 模組中匯入三種文件載入器 。用TextLoader來載入純文字(.txt) 檔案，用來載入 PDF 檔案
# 用UnstructuredWordDocumentLoader來載入 Word (.docx) 檔案，並支援複雜格式的解析
from langchain.text_splitter import RecursiveCharacterTextSplitter
# 從langchain 套件中匯入文字切割工具

from langchain.embeddings import HuggingFaceEmbeddings

# 匯入 Hugging Face 的向量嵌入模型（embeddings）

from langchain.vectorstores import FAISS

# 匯入 FAISS 向量資料庫工具

### 3. 依 e5 建議加入
自訂支援 E5 的 embedding 模型（加上 "passage:" / "query:" 前綴）

In [ ]:
# 自訂一個嵌入模型

from langchain.embeddings import HuggingFaceEmbeddings

class CustomE5Embedding(HuggingFaceEmbeddings):

# 定義一個新的類別 CustomE5Embedding，繼承自 HuggingFaceEmbeddings，用來擴充功能。

    def embed_documents(self, texts):
        texts = [f"passage: {t}" for t in texts]
        return super().embed_documents(texts)

    def embed_query(self, text):
        return super().embed_query(f"query: {text}")

### 4. 載入文件

In [ ]:
folder_path = upload_dir  # 設定要讀取檔案的資料夾路徑（這裡使用之前定義的 uploaded_docs 資料夾）
documents = []  # 建立一個空的列表，用來儲存所有讀取進來的文件內容

# 遍歷資料夾中所有檔案

for file in os.listdir(folder_path):
    path = os.path.join(folder_path, file)
    # 根據檔案副檔名使用對應的文件載入器
    if file.endswith(".txt"):
        loader = TextLoader(path) #如果是.txt檔案，就使用TextLoader 載入
    elif file.endswith(".pdf"):
        loader = PyPDFLoader(path) # 如果是.pdf檔案，就使用 PyPDFLoader 載入
    elif file.endswith(".docx"):
        loader = UnstructuredWordDocumentLoader(path) #如果是.docx 檔案，就使用 Word 檔解析器
    else:
        continue
    documents.extend(loader.load())

### 5. 建立向量資料庫

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# 每個切割後的段落最大長度為500字元(包括空白與標點)
# 每個段落之間會重疊 100 字元，以保留語意連貫性)


# 將先前讀取進來的 documents 切割成多個小段落
split_docs = splitter.split_documents(documents)

In [ ]:
embedding_model = CustomE5Embedding(model_name="intfloat/multilingual-e5-small")
vectorstore = FAISS.from_documents(split_docs, embedding_model)

### 6. 儲存向量資料庫

In [ ]:
vectorstore.save_local("faiss_db")

In [ ]:
!zip -r faiss_db.zip faiss_db

In [ ]:
print("✅ 壓縮好的向量資料庫已儲存為 'faiss_db.zip'，請下載此檔案備份。")